Basic SoftMax multiclass regression with default (3, 3) parameter FV on only the MFCC features.

In [1]:
import os
import scipy.io as spio
import numpy as np
from os.path import join, isdir, isfile

DATA_DIR = '../data'
# 'file_name' 'class_name' 'class' 'eng' 'mfc' 'chroma' 't'
# 'keystrength' 'brightness' 'zerocross' 'roughness' 'inharmonic' 'hcdf'
#

In [2]:
# In-python MFC extraction (may not be necessary if we use MATLAB-only)
X = []
for basename in os.listdir(DATA_DIR):
    path = join(DATA_DIR, basename)
    if not isdir(path): continue
    def loadmfc(base): return spio.loadmat(join(path, base))['DAT']['mfc'][0,0]
    mfc = np.concatenate([loadmfc(i).shape for i in os.listdir(path)])
    X.append([basename, mfc])
[classes, X] = zip(*X)
[len(a) for a in X]
# TODO: would be nice to have this fully in python.

[200, 200, 200, 200, 200, 200, 200, 200, 200, 200]

In [3]:
# This is technically wrong, since we're doing the FV computation on the whole dataset
FVs = join(DATA_DIR, 'FV.mat')
LBs = join(DATA_DIR, 'LB.mat')
if not isfile(FVs) or not isfile(LBs):
    print('Generating Fisher Vectors')
    !matlab -nodisplay -nosplash -nodesktop -r "run('../tools/FV_concat.m');exit;"
else:
    print('Using existing FVs')

mfcc = np.transpose(spio.loadmat(FVs)['FV'])
labels = spio.loadmat(LBs)['LB'][0]
N = mfcc.shape[1]
p = np.random.permutation(N)
mfcc, labels = mfcc[p], labels[p]
print('mfcc', mfcc.shape, 'labels', labels.shape)

Using existing FVs
mfcc (960, 960) labels (960,)


In [4]:
trN = int(0.8 * N)
teN = N - trN
trX, trY = mfcc[:trN], labels[:trN]
teX, teY = mfcc[-teN:], labels[-teN:]

In [16]:
# Random-restart softmax
from sklearn.linear_model import LogisticRegressionCV
import warnings
warnings.filterwarnings('ignore')

reg = LogisticRegressionCV(multi_class='multinomial', solver='lbfgs', max_iter=5000, n_jobs=8)
reg.fit(trX, trY)
reg.score(teX, teY)

0.68229166666666663